## Hardnet

Real-time semantic segmentation.

https://github.com/PingoLH/Pytorch-HarDNet

https://paperswithcode.com/paper/hardnet-a-low-memory-traffic-network#code

In [46]:
import torch
import urllib
from PIL import Image
from torchvision import transforms
import time
import numpy as np

In [13]:
img_f = '/home/sharif/Downloads/public_place.jpg'

In [71]:
torch.hub.list('PingoLH/Pytorch-HarDNet')

Using cache found in /home/sharif/.cache/torch/hub/PingoLH_Pytorch-HarDNet_master


['hardnet39ds', 'hardnet68', 'hardnet68ds', 'hardnet85']

In [72]:
model = torch.hub.load('PingoLH/Pytorch-HarDNet', 'hardnet68ds', pretrained=True)
model.eval()

Using cache found in /home/sharif/.cache/torch/hub/PingoLH_Pytorch-HarDNet_master
Downloading: "https://ping-chao.com/hardnet/hardnet68ds-632474d2.pth" to /home/sharif/.cache/torch/hub/checkpoints/hardnet68ds-632474d2.pth


ImageNet pretrained weights for HarDNet68DS is loaded


HarDNet(
  (base): ModuleList(
    (0): ConvLayer(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU6(inplace=True)
    )
    (1): ConvLayer(
      (conv): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (norm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU6(inplace=True)
    )
    (2): DWConvLayer(
      (dwconv): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=64, bias=False)
      (norm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (3): HarDBlock(
      (layers): ModuleList(
        (0): CombConvLayer(
          (layer1): ConvLayer(
            (conv): Conv2d(64, 14, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (norm): BatchNorm2d(14, eps=1e-05, momentum=0.1, affine=True, trac

In [58]:
img = Image.open(img_f)

In [59]:
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0) 

In [60]:
input_batch.shape

torch.Size([1, 3, 224, 224])

In [61]:
torch.cuda.is_available()

True

In [62]:
if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model.to('cuda')

In [63]:
with torch.no_grad():
    for _ in range(10):
        s = time.time()
        output = model(input_batch)
        e = time.time()

        print(f'FPS: {1 / (e-s)}')

FPS: 83.83410285622914
FPS: 72.6739439304155
FPS: 98.78247762600094
FPS: 102.27265855502182
FPS: 102.11579101134538
FPS: 104.50749987541735
FPS: 111.49429809404822
FPS: 110.79920750231145
FPS: 108.94579079976103
FPS: 107.45808567329371


In [64]:
output.shape

torch.Size([1, 1000])

In [40]:
output = torch.nn.functional.softmax(output[0], dim=0)

In [42]:
output[0]

tensor(1.9491e-08, device='cuda:0')

In [48]:
pred = np.squeeze(output.data.max(1)[1].cpu().numpy(), axis=0)